In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [189]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

In [190]:
y = df['label']
X = df.drop(['label'], axis=1)

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)

In [192]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [193]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [194]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [195]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.output_layer(out)
        return out

In [196]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [197]:
model = NeuralNetwork().to(device)

In [198]:
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [199]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)
    
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss}')
        print(f'current: {current}')

In [200]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f'test_loss: {test_loss}')
    print(f'correct: {correct}')

In [201]:
epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}")
    train_loop(train_dataloader,  model, loss, optimizer)
    test_loop(test_dataloader, model, loss)

Epoch 1
loss: 1.0277338027954102
current: 0
loss: 1.2351129055023193
current: 10
loss: 1.2621263265609741
current: 20
loss: 1.148000955581665
current: 30
loss: 1.1772562265396118
current: 40
loss: 1.14656662940979
current: 50
loss: 1.125928282737732
current: 60
loss: 1.036603331565857
current: 70
loss: 1.1560183763504028
current: 80
loss: 1.0907305479049683
current: 90
loss: 0.9591138958930969
current: 100
loss: 1.2175378799438477
current: 22
test_loss: 1.0744141340255737
correct: 0.34210526315789475
Epoch 2
loss: 1.1707463264465332
current: 0
loss: 1.0806280374526978
current: 10
loss: 1.1153770685195923
current: 20
loss: 0.9962830543518066
current: 30
loss: 1.0632169246673584
current: 40
loss: 1.059303879737854
current: 50
loss: 1.0241563320159912
current: 60
loss: 1.0463213920593262
current: 70
loss: 1.0192375183105469
current: 80
loss: 0.9962997436523438
current: 90
loss: 1.0191047191619873
current: 100
loss: 1.082967758178711
current: 22
test_loss: 1.019709661602974
correct: 0.4736